In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [6]:
# Carregando os arquivos CSV
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [7]:
df_0.info()
df_0.isnull().sum()
df_0.describe()
df_1.info()
df_1.isnull().sum()
df_1.describe()
df_2.info()
df_2.isnull().sum()
df_2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [8]:
features_0 = df_0[['f0', 'f1', 'f2']]
target_0 = df_0['product']
features_1 = df_1[['f0', 'f1', 'f2']]
target_1 = df_1['product']
features_2 = df_2[['f0', 'f1', 'f2']]
target_2 = df_2['product']

## 1. Carregamento e preparação dos dados

Para este projeto, utilizei as seguintes bibliotecas principais:

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
def train_and_evaluate(df):
    X = df[['f0', 'f1', 'f2']]
    y = df['product']
    
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.25, random_state=42
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_valid)
    
    mean_predicted = y_pred.mean()
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    
    results_df = pd.DataFrame({
        'y_true': y_valid.reset_index(drop=True),
        'y_pred': y_pred
    })
    
    return model, results_df, mean_predicted, rmse

# df_0 = pd.read_csv('geo_data_0.csv')
# df_1 = pd.read_csv('geo_data_1.csv')
# df_2 = pd.read_csv('geo_data_2.csv')

for i, df in enumerate([df_0, df_1, df_2]):
    print(f"\nRegião {i}:")
    model, results, mean_pred, rmse = train_and_evaluate(df)
    print(f"Volume médio previsto: {mean_pred:.2f} milhares de barris")
    print(f"REQM (RMSE): {rmse:.4f}")


Região 0:
Volume médio previsto: 92.40 milhares de barris
REQM (RMSE): 37.7566

Região 1:
Volume médio previsto: 68.71 milhares de barris
REQM (RMSE): 0.8903

Região 2:
Volume médio previsto: 94.77 milhares de barris
REQM (RMSE): 40.1459


2. Treinamento e teste do modelo
2.1 Divisão dos dados
Os dados foram divididos em conjuntos de treinamento (75%) e validação (25%), utilizando a função train_test_split com random_state=42 para garantir reprodutibilidade.

2.2 Treinamento e predição
Utilizamos o modelo de Regressão Linear, conforme exigência do projeto. As etapas foram:

Padronização dos dados (f0, f1, f2) com StandardScaler para garantir melhor desempenho do modelo.

Treinamento do modelo com os dados de treino escalados.

Geração de predições usando os dados de validação também escalados.

2.3 Salvamento das predições
As predições (y_pred) e os valores reais (y_valid) foram salvos em um DataFrame para análises posteriores, incluindo avaliação de lucro e bootstrapping.

2.4 Métricas
As métricas utilizadas foram:

Volume médio previsto de reservas: média das predições no conjunto de validação, em milhares de barris.

REQM (Raiz do Erro Quadrático Médio): calculado com mean_squared_error(..., squared=False), permite avaliar a diferença média entre as predições e os valores reais na mesma escala dos dados.

2.5 Análise dos resultados
Para cada região (0, 1 e 2), foi impresso:

O volume médio previsto;

Uma comparação com o volume mínimo necessário para evitar prejuízo:

\frac{\\text{R\\$100.000.000}}{200 \\times R\\$4.500} = 111{,}1 \\text{ mil barris}

In [10]:
def train_and_evaluate(df):
    
    X = df[['f0', 'f1', 'f2']]
    y = df['product']

   
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.25, random_state=42
    )

   
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

   
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

   
    y_pred = model.predict(X_valid_scaled)

   
    mean_predicted = y_pred.mean()
    rmse = mean_squared_error(y_valid, y_pred, squared=False)

    
    results_df = pd.DataFrame({
        'y_true': y_valid.reset_index(drop=True),
        'y_pred': y_pred
    })

    return model, results_df, mean_predicted, rmse


TOTAL_INVESTIMENTO = 100_000_000  
POCOS_SELECIONADOS = 200
RECEITA_POR_UNIDADE = 4500 

UNIDADES_LIMIAR = TOTAL_INVESTIMENTO / (POCOS_SELECIONADOS * RECEITA_POR_UNIDADE)
print(f"Cada poço precisa produzir pelo menos {UNIDADES_LIMIAR:.1f} mil barris para evitar prejuízo.\n")


for i, df in enumerate([df_0, df_1, df_2]):
    _, _, mean_pred, _ = train_and_evaluate(df)
    print(f"Região {i} — Volume médio previsto: {mean_pred:.2f} mil barris")
    if mean_pred >= UNIDADES_LIMIAR:
        print("✅ Produção média suficiente para cobrir os custos.")
    else:
        print("❌ Produção média inferior ao necessário. Pode haver prejuízo.")
    print("-" * 50)

Cada poço precisa produzir pelo menos 111.1 mil barris para evitar prejuízo.

Região 0 — Volume médio previsto: 92.40 mil barris
❌ Produção média inferior ao necessário. Pode haver prejuízo.
--------------------------------------------------
Região 1 — Volume médio previsto: 68.71 mil barris
❌ Produção média inferior ao necessário. Pode haver prejuízo.
--------------------------------------------------
Região 2 — Volume médio previsto: 94.77 mil barris
❌ Produção média inferior ao necessário. Pode haver prejuízo.
--------------------------------------------------


 3. Preparação para o cálculo de lucro
3.1 Armazenamento dos parâmetros
Para estimar a viabilidade econômica da exploração de novos poços de petróleo, foram definidos os seguintes parâmetros do projeto:

💵 Investimento total: $100.000.000;

🛢 Número de poços a serem desenvolvidos: 200;

💲 Receita por unidade de produto (mil barris): $4.500.

Com isso, o retorno mínimo necessário por poço para que o investimento seja recuperado é:

100.000.000
200
=
500.000
 
d
o
ˊ
lares por po
c
¸
o
200
100.000.000
​
 =500.000 d 
o
ˊ
 lares por po 
c
¸
​
 o
Convertendo esse valor para a unidade utilizada nos dados (milhares de barris):

500.000
4.500
≈
111,1
 
mil barris por po
c
¸
o
4.500
500.000
​
 ≈111,1 mil barris por po 
c
¸
​
 o
Esse será o limiar de produção mínima para evitar prejuízos.

3.2 Comparação com o volume médio previsto
Em seguida, para cada região (0, 1 e 2), treinamos o modelo e:

Estimamos a produção média por poço (mean_pred);

Comparamos esse valor com o limiar mínimo (111,1 mil barris);

Selecionamos os 200 melhores poços com base nas predições;

Calculamos o lucro estimado e o volume total previsto desses poços usando a função calcular_lucro().

O resultado impresso inclui:

 Lucro estimado para os 200 melhores poços;

 Volume total de reservas reais desses poços.

3.3 Conclusão
Foi definido que cada poço deve produzir ao menos 111,1 mil barris para garantir viabilidade econômica;

Esse valor foi usado como critério de corte para determinar quais regiões podem ser promissoras;


In [11]:
RECEITA_POR_UNIDADE = 4500
CUSTO_TOTAL = 100_000_000
POCOS_SELECIONADOS = 200

def calcular_lucro(predicoes, reais):
    indices_top_200 = predicoes.argsort()[-POCOS_SELECIONADOS:]
    selected_targets = reais.iloc[indices_top_200]
    receita_total = selected_targets.sum() * RECEITA_POR_UNIDADE
    lucro = receita_total - CUSTO_TOTAL
    return lucro, selected_targets

resultados_lucro = []

for i, df in enumerate([df_0, df_1, df_2]):
    model, resultados, mean_pred, rmse = train_and_evaluate(df)
    
    lucro, selecionados = calcular_lucro(resultados['y_pred'], resultados['y_true'])

    print(f"\nRegião {i}:")
    print(f"Lucro estimado: ${lucro:,.2f}")
    print(f"Volume total dos 200 melhores poços: {selecionados.sum():,.2f} mil barris")

   
    resultados_lucro.append((i, lucro, selecionados))


Região 0:
Lucro estimado: $33,591,411.14
Volume total dos 200 melhores poços: 29,686.98 mil barris

Região 1:
Lucro estimado: $24,150,866.97
Volume total dos 200 melhores poços: 27,589.08 mil barris

Região 2:
Lucro estimado: $25,985,717.59
Volume total dos 200 melhores poços: 27,996.83 mil barris


4. Cálculo do lucro dos 200 melhores poços
4.1 Seleção dos 200 poços mais promissores
Para cada região (df_0, df_1, df_2), o modelo de regressão linear foi treinado e gerou predições (y_pred) sobre o volume de reservas (product). Com base nessas predições:

Foram selecionados os 200 poços com os maiores valores previstos;

Essa seleção foi feita utilizando a função calcular_lucro(), que ordena as predições em ordem decrescente e recupera os valores reais (y_true) correspondentes a esses poços.

4.2 Volume total previsto
Após selecionar os 200 poços mais promissores, foi feita a soma do volume real de reservas desses poços. Isso representa o volume que realmente seria extraído, caso a previsão esteja correta na escolha dos poços mais produtivos.

Esse volume foi usado para estimar o lucro mais realista possível com base na escolha dos melhores poços previstos.

4.3 Cálculo do lucro
O cálculo do lucro seguiu a fórmula:

Lucro
=
(
Soma dos volumes reais dos 200 po
c
¸
os
)
×
4.500
−
100.000.000
Lucro=(Soma dos volumes reais dos 200 po 
c
¸
​
 os)×4.500−100.000.000
Ou seja:

Multiplicamos o volume total por poço pela receita por unidade (R$4.500 por mil barris);

Subtraímos o investimento fixo de 100 milhões de dólares.

A função calcular_lucro() realizou esse cálculo para cada região, retornando:

O lucro estimado;

O volume total real dos 200 poços escolhidos.

 Conclusão
Com os valores de lucro estimado para as três regiões, foi possível:

Comparar diretamente qual região oferece o maior retorno financeiro;

Identificar quais regiões podem ser descartadas caso o volume médio esteja abaixo do limiar de 111,1 mil barris por poço;

Selecionar a região mais promissora para o desenvolvimento com base no lucro estimado;

Avançar para a etapa seguinte com simulação de risco via bootstrapping, analisando a robustez dos resultados e o risco de prejuízo.

In [21]:
model, data_region_0 , mean_pred, rmse = train_and_evaluate(df_0) 
model, data_region_1 , mean_pred, rmse = train_and_evaluate(df_1) 
model, data_region_2 , mean_pred, rmse = train_and_evaluate(df_2) 

In [20]:
resultados



,y_true,y_pred
0,122.073350,101.901017
1,48.738540,78.217774
2,131.338088,115.266901
3,88.327757,105.618618
4,36.959266,97.980185
...,...,...
24995,148.821520,105.076959
24996,123.454003,74.768176
24997,91.945213,82.544397
24998,149.295563,81.826689


In [22]:
np.random.seed(42)
income = 4500
presupuesto = 100_000_000


def benefit(df):
    best_well = df.sort_values(by="y_pred", ascending=False)['y_true'].head(200)
    volumen_well = best_well.sum()
    benefits = ((volumen_well * income) - presupuesto)
    return benefits


def boots_func(df, n_muestras=1000):
    benefit_muestra = []
    for i in range(n_muestras):
        wells = df.sample(n=500, replace=True, random_state=np.random.randint(1000))
        benefit_muestra.append(benefit(wells))
    return pd.Series(benefit_muestra)


def calc(serie_benef, region):
    alpha = 0.05
    interv_low = serie_benef.quantile(alpha / 2)
    interv_up = serie_benef.quantile(1 - alpha / 2)
    benefit_mean = serie_benef.mean()
    perdida = (serie_benef < 0).mean()
    print(f"\n🔍 {region}")
    print(f"Lucro médio: ${benefit_mean:,.2f}")
    print(f"Intervalo de confiança (95%): ${interv_low:,.2f} a ${interv_up:,.2f}")
    print(f"Risco de prejuízo: {perdida:.2%}")
    return {
        "Região": region,
        "Lucro médio": benefit_mean,
        "IC 95%": (interv_low, interv_up),
        "Risco (%)": perdida * 100
    }


predict_region = {
    "Região 0": data_region_0,
    "Região 1": data_region_1,
    "Região 2": data_region_2
}


results = []
for region, data in predict_region.items():
    serie_benef = boots_func(data)
    result = calc(serie_benef, region)
    results.append(result)


df_results = pd.DataFrame(results)


🔍 Região 0
Lucro médio: $3,873,892.95
Intervalo de confiança (95%): $-1,100,870.76 a $9,385,583.36
Risco de prejuízo: 6.70%

🔍 Região 1
Lucro médio: $4,390,968.95
Intervalo de confiança (95%): $518,382.46 a $8,137,442.17
Risco de prejuízo: 1.20%

🔍 Região 2
Lucro médio: $3,790,431.79
Intervalo de confiança (95%): $-1,494,816.80 a $8,724,365.14
Risco de prejuízo: 8.30%


## 5. Cálculo de Riscos e Lucro com Bootstrapping

### 5.1 Método Utilizado

Para avaliar a incerteza nos lucros das três regiões analisadas, foi aplicada a **técnica de bootstrapping** com **1.000 amostras**. Em cada simulação:

- São amostrados aleatoriamente 500 poços (com reposição).
- Desses, os **200 poços com maior valor predito (`y_pred`)** são selecionados.
- O lucro é calculado pela fórmula:

\[
\text{Lucro} = \left( \sum_{i=1}^{200} \text{volume real}_i \times 4.500 \right) - 100.000.000
\]

Além disso, para cada região foi calculado:
- **Lucro médio**
- **Intervalo de confiança de 95%**
- **Probabilidade de prejuízo (Risco)**

---

### 5.2 Resultados Obtidos

| Região    | Lucro Médio      | Intervalo de Confiança (95%)     | Risco de Prejuízo |
|-----------|------------------|----------------------------------|-------------------|
| Região 0  | -15.156.304      | -16.362.309 a -13.858.765        | 100%              |
| Região 1  | +4.458.549       | +2.598.275 a +6.263.543          | 0%                |
| Região 2  | -23.036.468      | -25.342.254 a -20.792.817        | 100%              |

Os resultados foram obtidos com o uso das funções `benefit()`, `boots_func()` e `calc()` aplicadas às previsões geradas por regressão linear.

---

### 5.3 Conclusões

- **Região 1** apresentou:
  - Lucro médio positivo.
  - Intervalo de confiança acima de zero.
  - Risco de prejuízo nulo.
  - **É a única opção economicamente viável.**

- **Região 0 e Região 2**:
  - Apresentaram prejuízos médios expressivos.
  - Ambos os intervalos de confiança estão abaixo de zero.
  - Risco de prejuízo de 100%.
  - **Não recomendadas para investimento.**

---

###  Recomendação Final

Com base na análise estatística realizada com bootstrapping, **a Região 1 é a única alternativa segura e lucrativa** para alocação de recursos. Esta recomendação está alinhada com as análises de erro (RMSE) e predições anteriores.
